## Machine Learning Project on UCI Parkinsons Telemonitoring Data Set

Importación de librerías

In [ ]:
#!pip install qgrid

In [1]:
from __future__ import division

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import numpy.matlib as matlib

from numpy import random
from numpy import round
import math

import pandas as pd
import qgrid

import scipy as sc
from scipy.spatial import distance
from scipy import stats

#algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")

Cargamos la base de datos

In [2]:
#Cargamos la bd que está en un archivo .data y ahora la podemos manejar de forma matricial
db = np.loadtxt('DB/parkinsons_updrs.data', delimiter=',', skiprows=1)  # Assuming ',' delimiter

#X: Toma todas las filas (muestras) y las columnas 6-21 (características)
X = db[:,6:22]
#Y: Toma todas las filas y la columna 4, corresponde a la salida de la regresión
Y = db[:,4]
#G: Toma todas las filas y la columna 0, corresponde a la asociación en grupos del dataset
G = db[:,0]

In [3]:
print(X.shape)
print(Y.shape)
print(G.shape)

(5875, 16)
(5875,)
(5875,)


Medidas de error

In [4]:
#Mean Absolute Percentage Error
def MAPE(Y, Y_est):
    N = np.size(Y)
    mape = (1/N)*np.sum(abs((Y_est.reshape(N,1) - Y.reshape(N,1))/Y.reshape(N,1)))
    return mape

#### Regresión Lineal Múltiple

In [6]:
# MAE, R2, RMSE

from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score, mean_squared_error

# Normalizamos los datos
scaler = preprocessing.StandardScaler().fit(X)
X_norm = scaler.transform(X)

random.seed(19680801)

iterations = 10
gss = GroupShuffleSplit(n_splits=iterations, train_size=.7)
index = gss.split(X, Y, G)

# modelo
model = Pipeline([('poly', PolynomialFeatures()), ('linear', LinearRegression(fit_intercept=False))])
#print(model.get_params().keys())

parameters = {'poly__degree': [1,2,3,4]}

# métricas de error
# greater_is_better=True by default
mae = make_scorer(mean_absolute_error)
r2 = make_scorer(r2_score)
rmse = make_scorer(mean_squared_error, squared=False)

scores = {'mae':mae,'r2':r2,'rmse':rmse}

# Run the grid search
grid_obj = GridSearchCV(model, parameters, scoring=scores, cv=index, return_train_score=True, refit=False)
grid_obj = grid_obj.fit(X_norm, Y)

In [7]:
outcomes = pd.DataFrame(grid_obj.cv_results_)
outcomes = outcomes[['params', 'mean_train_mae', 'mean_train_r2', 'mean_train_rmse',
            'mean_test_mae', 'std_test_mae', 'mean_test_r2', 'std_test_r2', 'mean_test_rmse', 'std_test_rmse']]
outcomes

,params,mean_train_mae,mean_train_r2,mean_train_rmse,mean_test_mae,std_test_mae,mean_test_r2,std_test_r2,mean_test_rmse,std_test_rmse
0,{'poly__degree': 1},6.351467,0.137742,7.524888,7.113906,0.258448,-1.907575e-01,1.597205e-01,8.577520e+00,4.485203e-01
1,{'poly__degree': 2},5.461242,0.316153,6.700819,8.240104,1.386231,-2.519604e+00,4.355412e+00,1.299009e+01,7.007837e+00
2,{'poly__degree': 3},4.150653,0.560697,5.368819,42.478125,51.503028,-1.675949e+04,3.889956e+04,5.516074e+02,8.421256e+02
3,{'poly__degree': 4},0.000259,1.000000,0.000496,96705.654730,112886.283714,-7.763870e+10,8.642567e+10,1.706706e+06,1.352049e+06


#### Ventana de Parzen

In [5]:
def kernel_gaussiano(x):
    return (np.exp((-0.5)*x**2))

def Nadaraya_Watson(X_train, Y_train, X_val, h): # h equivale al ancho de la ventana o kernel
    Nval = len(X_val)
    Ntrain = len(X_train)
    Y_val = np.zeros(Nval)
    
    for i in range(Nval):
        muestra = X_val[i,:]
        numerador = 0
        denominador = 0
        for j in range(Ntrain):
            distancia = distance.euclidean(muestra, X_train[j])/h
            peso = kernel_gaussiano(distancia)
            numerador += peso * Y_train[j]
            denominador += peso
        Y_val[i] = numerador/denominador
    #Se retorna un vector que contiene las predicciones para cada una de las muestras en X_val, en el mismo orden.      
    return Y_val

In [11]:
# MAE, R2

from sklearn.model_selection import GroupShuffleSplit
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error, r2_score

def executePARZEN_NW(h):
    
    iterations = 5
    errorMAE = np.zeros(iterations)
    errorR2 = np.zeros(iterations)
    
    random.seed(19680801)
    gss = GroupShuffleSplit(n_splits=iterations, train_size=.7)
    j=0
    for train_idx, test_idx in gss.split(X, Y, G):
        Xtrain = X[train_idx,:]
        Ytrain = Y[train_idx]
        Xtest = X[test_idx,:]
        Ytest = Y[test_idx]

        #Normalizamos los datos
        media = np.mean(Xtrain)
        desvia = np.std(Xtrain)
        Xtrain = sc.stats.stats.zscore(Xtrain)
        Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)

        Yest = Nadaraya_Watson(Xtrain, Ytrain, Xtest, h)

        #Evaluamos las predicciones del modelo con los datos de test
        errorMAE[j] = mean_absolute_error(Ytest, Yest)
        errorR2[j] = r2_score(Ytest, Yest)
        j += 1

    mae = np.mean(errorMAE)
    std_mae = np.std(errorMAE)
    r2 = np.mean(errorR2)
    std_r2 = np.std(errorR2)
    
    return(round(mae,4), round(std_mae,4), round(r2,4), round(std_r2,4))


In [13]:
df_types = pd.DataFrame({
    'Ancho del kernel' : pd.Series(['0.5', '1', '2', '5','10'])})
df_types["MAE"] = ""
df_types["std_MAE"] = ""
df_types["R2"] = ""
df_types["std_R2"] = ""
df_types.set_index(['Ancho del kernel'], inplace=True)

valoresH = np.array([0.5,1,2,5,10])
for i in range(np.size(valoresH)):
    mae, std_mae, r2, std_r2  = executePARZEN_NW(valoresH[i])
    df_types["MAE"][i] = mae
    df_types["std_MAE"][i] = std_mae
    df_types["R2"][i] = r2
    df_types["std_R2"][i] = std_r2
    
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [14]:
qgrid_widget.get_changed_df()

,MAE,std_MAE,R2,std_R2
Ancho del kernel,,,,
0.5,10.3501,1.8655,-1.5605,0.9141
1,7.5742,1.0014,-0.3956,0.4086
2,7.0959,0.8243,-0.1634,0.2377
5,7.018,0.7643,-0.1185,0.1969
10,6.9963,0.7396,-0.1074,0.1825


#### Redes Neuronales Artificales

In [ ]:
# MAE, R2, RMSE

from sklearn.model_selection import GroupShuffleSplit
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score, mean_squared_error

# Normalizamos los datos
scaler = preprocessing.StandardScaler().fit(X)
X_norm = scaler.transform(X)

random.seed(19680801)

iterations = 10
gss = GroupShuffleSplit(n_splits=iterations, train_size=.7)
index = gss.split(X, Y, G)

model = MLPRegressor(activation='tanh')

parameters = {'hidden_layer_sizes': [(8),(16),(32),(8,8),(16,16),(32,32),(8,8,8),(16,16,16),(32,32,32)],
             'max_iter': [500,1000,1500]}

# métricas de error
# greater_is_better=True by default
mae = make_scorer(mean_absolute_error)
r2 = make_scorer(r2_score)
rmse = make_scorer(mean_squared_error, squared=False)

scores = {'mae':mae,'r2':r2,'rmse':rmse}

# Run the grid search
grid_obj = GridSearchCV(model, parameters, scoring=scores, cv=index, return_train_score=True, refit=False)
grid_obj = grid_obj.fit(X_norm, Y)

In [12]:
outcomes = pd.DataFrame(grid_obj.cv_results_)
outcomes = outcomes[['params', 'mean_train_mae', 'mean_train_r2', 'mean_train_rmse',
            'mean_test_mae', 'std_test_mae', 'mean_test_r2', 'std_test_r2', 'mean_test_rmse', 'std_test_rmse']]
outcomes

,params,mean_train_mae,mean_train_r2,mean_train_rmse,mean_test_mae,std_test_mae,mean_test_r2,std_test_r2,mean_test_rmse,std_test_rmse
0,"{'hidden_layer_sizes': 8, 'max_iter': 500}",5.562746,0.279462,6.875116,7.547490,0.465845,-0.358031,0.233509,9.136517,0.529503
1,"{'hidden_layer_sizes': 8, 'max_iter': 1000}",5.406609,0.314536,6.705013,7.631354,0.584386,-0.389432,0.267782,9.233823,0.663682
2,"{'hidden_layer_sizes': 8, 'max_iter': 1500}",5.374483,0.318367,6.685240,7.670587,0.564504,-0.389749,0.238831,9.247882,0.600639
3,"{'hidden_layer_sizes': 16, 'max_iter': 500}",5.382395,0.318178,6.684793,7.587403,0.556264,-0.369906,0.240422,9.178715,0.594459
4,"{'hidden_layer_sizes': 16, 'max_iter': 1000}",5.141643,0.366153,6.447493,7.737584,0.532623,-0.417830,0.217151,9.353877,0.584137
5,"{'hidden_layer_sizes': 16, 'max_iter': 1500}",5.022856,0.389383,6.331027,7.852918,0.522782,-0.462418,0.198323,9.509979,0.574289
6,"{'hidden_layer_sizes': 32, 'max_iter': 500}",5.136924,0.371123,6.423922,7.637718,0.519401,-0.381516,0.215259,9.230038,0.557211
7,"{'hidden_layer_sizes': 32, 'max_iter': 1000}",4.770705,0.442188,6.047095,7.847101,0.458356,-0.460270,0.201002,9.498697,0.525808
8,"{'hidden_layer_sizes': 32, 'max_iter': 1500}",4.628095,0.470133,5.891623,7.995462,0.495639,-0.525111,0.231852,9.698625,0.572901
9,"{'hidden_layer_sizes': (8, 8), 'max_iter': 500}",5.020207,0.389342,6.329784,7.705481,0.456041,-0.427920,0.218260,9.381973,0.484795


#### Random Forest

In [ ]:
# MAE, R2
from sklearn.model_selection import GroupShuffleSplit
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score

# Normalizamos los datos
scaler = preprocessing.StandardScaler().fit(X)
X_norm = scaler.transform(X)

random.seed(19680801)

iterations = 10
gss = GroupShuffleSplit(n_splits=iterations, train_size=.7)
index = gss.split(X, Y, G)

model = RandomForestRegressor()

parameters = {'n_estimators': [5,10,20,50,100],
             'max_features': [5,10,16]}

# greater_is_better=True by default
mae = make_scorer(mean_absolute_error)
r2 = make_scorer(r2_score)

scores = {'mae':mae,'r2':r2}

# Run the grid search
grid_obj = GridSearchCV(model, parameters, scoring=scores, cv=index, return_train_score=True, refit=False)
grid_obj = grid_obj.fit(X_norm, Y)


In [9]:
outcomes = pd.DataFrame(grid_obj.cv_results_)
outcomes = outcomes[['params', 'mean_train_mae', 'mean_train_r2',
            'mean_test_mae', 'std_test_mae', 'mean_test_r2', 'std_test_r2']]
outcomes

,params,mean_train_mae,mean_train_r2,mean_test_mae,std_test_mae,mean_test_r2,std_test_r2
0,"{'max_features': 5, 'n_estimators': 5}",2.154013,0.842394,7.736161,0.342465,-0.440328,0.181880
1,"{'max_features': 5, 'n_estimators': 10}",2.004675,0.880579,7.567269,0.367710,-0.361952,0.166379
2,"{'max_features': 5, 'n_estimators': 20}",1.893591,0.902065,7.403156,0.367118,-0.297528,0.160270
3,"{'max_features': 5, 'n_estimators': 50}",1.820054,0.914683,7.324262,0.330162,-0.265431,0.144835
4,"{'max_features': 5, 'n_estimators': 100}",1.802447,0.918256,7.320253,0.334875,-0.261077,0.145253
5,"{'max_features': 10, 'n_estimators': 5}",2.075158,0.847546,7.809514,0.303777,-0.464814,0.170340
6,"{'max_features': 10, 'n_estimators': 10}",1.947880,0.885185,7.631766,0.313510,-0.388329,0.167694
7,"{'max_features': 10, 'n_estimators': 20}",1.840819,0.905582,7.465276,0.329436,-0.324875,0.149550
8,"{'max_features': 10, 'n_estimators': 50}",1.783124,0.916239,7.410669,0.360195,-0.300313,0.143300
9,"{'max_features': 10, 'n_estimators': 100}",1.756294,0.920376,7.413100,0.351831,-0.297298,0.146518


#### Máquinas de Vectores de Soporte

In [4]:
# MAE, R2, RMSE

from sklearn.model_selection import GroupShuffleSplit
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score, mean_squared_error

# Normalizamos los datos
scaler = preprocessing.StandardScaler().fit(X)
X_norm = scaler.transform(X)

random.seed(19680801)

iterations = 10
gss = GroupShuffleSplit(n_splits=iterations, train_size=.7)
index = gss.split(X, Y, G)

model = SVR()

parameters = {'kernel': ['linear', 'rbf'],
              'C': [0.01, 1, 10],
              'gamma': [0.01, 0.1, 1]}

#print(model.get_params().keys())

# métricas de error
# greater_is_better=True by default
mae = make_scorer(mean_absolute_error)
r2 = make_scorer(r2_score)
rmse = make_scorer(mean_squared_error, squared=False)

scores = {'mae':mae,'r2':r2,'rmse':rmse}

# Run the grid search
grid_obj = GridSearchCV(model, parameters, scoring=scores, cv=index, return_train_score=True, refit=False, verbose=12)
grid_obj = grid_obj.fit(X_norm, Y)

/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 19

Fitting 10 folds for each of 18 candidates, totalling 180 fits
[CV] C=0.01, gamma=0.01, kernel=linear ...............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  C=0.01, gamma=0.01, kernel=linear, mae=(train=6.334, test=7.165), r2=(train=0.031, test=0.038), rmse=(train=7.828, test=8.307), total=   1.5s
[CV] C=0.01, gamma=0.01, kernel=linear ...............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


...

[CV]  C=0.01, gamma=1, kernel=linear, mae=(train=6.753, test=6.346), r2=(train=0.082, test=0.027), rmse=(train=8.022, test=7.389), total=   0.6s
[CV] C=0.01, gamma=1, kernel=rbf .....................................
[CV]  C=0.01, gamma=1, kernel=rbf, mae=(train=6.701, test=7.297), r2=(train=0.009, test=0.006), rmse=(train=7.918, test=8.444), total=   1.2s
[CV] C=0.01, gamma=1, kernel=rbf .....................................
[CV]  C=0.01, gamma=1, kernel=rbf, mae=(train=6.657, test=7.461), r2=(train=0.009, test=-0.017), rmse=(train=8.021, test=8.361), total=   1.1s
[CV] C=0.01, gamma=1, kernel=rbf .....................................
[CV]  C=0.01, gamma=1, kernel=rbf, mae=(train=7.302, test=5.941), r2=(train=0.035, test=-0.031), rmse=(train=8.297, test=7.468), total=   1.2s
[CV] C=0.01, gamma=1, kernel=rbf .....................................
[CV]  C=0.01, gamma=1, kernel=rbf, mae=(train=7.059, test=6.493), r2=(train=0.028, test=-0.046), rmse=(train=8.250, test=7.713), total=   1.1s


[CV]  C=1, gamma=0.1, kernel=linear, mae=(train=6.142, test=7.825), r2=(train=0.126, test=-0.703), rmse=(train=7.562, test=10.192), total=   0.9s
[CV] C=1, gamma=0.1, kernel=linear ...................................
[CV]  C=1, gamma=0.1, kernel=linear, mae=(train=6.433, test=7.317), r2=(train=0.111, test=-0.484), rmse=(train=7.893, test=9.125), total=   0.8s
[CV] C=1, gamma=0.1, kernel=rbf ......................................
[CV]  C=1, gamma=0.1, kernel=rbf, mae=(train=5.094, test=7.055), r2=(train=0.325, test=0.010), rmse=(train=6.534, test=8.428), total=   1.2s
[CV] C=1, gamma=0.1, kernel=rbf ......................................
[CV]  C=1, gamma=0.1, kernel=rbf, mae=(train=5.299, test=7.928), r2=(train=0.268, test=-0.247), rmse=(train=6.895, test=9.261), total=   1.1s
[CV] C=1, gamma=0.1, kernel=rbf ......................................
[CV]  C=1, gamma=0.1, kernel=rbf, mae=(train=5.143, test=7.997), r2=(train=0.372, test=-0.672), rmse=(train=6.690, test=9.510), total=   1.1s


[CV]  C=10, gamma=0.01, kernel=linear, mae=(train=5.983, test=7.926), r2=(train=0.141, test=-0.514), rmse=(train=7.564, test=9.776), total=   1.7s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV]  C=10, gamma=0.01, kernel=linear, mae=(train=6.138, test=7.924), r2=(train=0.123, test=-0.793), rmse=(train=7.576, test=10.458), total=   1.6s
[CV] C=10, gamma=0.01, kernel=linear .................................
[CV]  C=10, gamma=0.01, kernel=linear, mae=(train=6.431, test=7.382), r2=(train=0.105, test=-0.531), rmse=(train=7.921, test=9.267), total=   2.1s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV]  C=10, gamma=0.01, kernel=rbf, mae=(train=5.620, test=6.954), r2=(train=0.222, test=0.043), rmse=(train=7.017, test=8.287), total=   1.1s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV]  C=10, gamma=0.01, kernel=rbf, mae=(train=5.654, test=8.063), r2=(train=0.184, test=-0.291), rmse=(train=7.278, test=9.422), t

[CV]  C=10, gamma=1, kernel=linear, mae=(train=6.475, test=6.649), r2=(train=0.046, test=-0.094), rmse=(train=8.014, test=7.787), total=   2.0s
[CV] C=10, gamma=1, kernel=linear ....................................
[CV]  C=10, gamma=1, kernel=linear, mae=(train=5.983, test=7.926), r2=(train=0.141, test=-0.514), rmse=(train=7.564, test=9.776), total=   1.9s
[CV] C=10, gamma=1, kernel=linear ....................................
[CV]  C=10, gamma=1, kernel=linear, mae=(train=6.138, test=7.924), r2=(train=0.123, test=-0.793), rmse=(train=7.576, test=10.458), total=   1.7s
[CV] C=10, gamma=1, kernel=linear ....................................
[CV]  C=10, gamma=1, kernel=linear, mae=(train=6.431, test=7.382), r2=(train=0.105, test=-0.531), rmse=(train=7.921, test=9.267), total=   1.9s
[CV] C=10, gamma=1, kernel=rbf .......................................
[CV]  C=10, gamma=1, kernel=rbf, mae=(train=2.133, test=7.230), r2=(train=0.771, test=-0.119), rmse=(train=3.804, test=8.961), total=   4.4

[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed:  5.4min finished


In [5]:
outcomes = pd.DataFrame(grid_obj.cv_results_)
outcomes = outcomes[['params', 'mean_train_mae', 'mean_train_r2', 'mean_train_rmse',
            'mean_test_mae', 'std_test_mae', 'mean_test_r2', 'std_test_r2', 'mean_test_rmse', 'std_test_rmse']]
outcomes

,params,mean_train_mae,mean_train_r2,mean_train_rmse,mean_test_mae,std_test_mae,mean_test_r2,std_test_r2,mean_test_rmse,std_test_rmse
0,"{'C': 0.01, 'gamma': 0.01, 'kernel': 'linear'}",6.378947,0.074481,7.796932,7.223411,0.583880,-0.191601,0.233549,8.570458,0.774820
1,"{'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'}",6.867655,0.000571,8.105998,7.168634,0.739670,-0.153300,0.215761,8.441721,0.782439
2,"{'C': 0.01, 'gamma': 0.1, 'kernel': 'linear'}",6.378947,0.074481,7.796932,7.223411,0.583880,-0.191601,0.233549,8.570458,0.774820
3,"{'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'}",6.755610,0.027656,7.993980,7.201606,0.682692,-0.156301,0.203188,8.456004,0.745043
4,"{'C': 0.01, 'gamma': 1, 'kernel': 'linear'}",6.378947,0.074481,7.796932,7.223411,0.583880,-0.191601,0.233549,8.570458,0.774820
5,"{'C': 0.01, 'gamma': 1, 'kernel': 'rbf'}",6.835743,0.010151,8.067303,7.160230,0.718521,-0.152179,0.205011,8.440372,0.755263
6,"{'C': 1, 'gamma': 0.01, 'kernel': 'linear'}",6.223547,0.095360,7.706775,7.487611,0.467114,-0.354864,0.253703,9.123859,0.705711
7,"{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}",6.128618,0.125013,7.577974,7.327706,0.528497,-0.233535,0.221235,8.718466,0.683757
8,"{'C': 1, 'gamma': 0.1, 'kernel': 'linear'}",6.223547,0.095360,7.706775,7.487611,0.467114,-0.354864,0.253703,9.123859,0.705711
9,"{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}",5.264087,0.305796,6.749162,7.455751,0.452823,-0.300730,0.207265,8.953434,0.535530
